In [1]:
import pandas as pd
import json
import sys
import datetime
import shutil
import os
import networkx as nx
import matplotlib.pyplot as plt
import datetime

from openai import OpenAI
from dotenv import load_dotenv
from google.cloud.exceptions import NotFound
from google.cloud import storage, bigquery

load_dotenv()

True

In [2]:
# NUEVA CONFIGURACIÓN ACTUALIZADA
REGION = "us-central1"  # Esta variable podría mantenerse igual, pero es bueno confirmarla.
PROJECT_ID = "noticias-450720"
BIGQUERY_DATASET = "news_articles" # Recomiendo un nombre de variable más descriptivo
TABLE = "articles"

In [3]:
!gcloud config set project $PROJECT_ID
!gcloud auth login # Cloud SDK
!gcloud auth application-default login


To update your Application Default Credentials quota project, use the `gcloud auth application-default set-quota-project` command.
Updated property [core/project].
ERROR: (gcloud.auth.login) unrecognized arguments:
  Cloud
  SDK
  To search the help text of gcloud commands, run:
  gcloud help -- SEARCH_TERMS
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=PymOOX8RWLJkI1wHfaoVIVpKqAiwKW&access_type=offline&code_challenge=0M93ajQEPgxMD5c1nyM780t8A8cgkPczcnLAaSGTGKs&code_challenge_method=S256


Credentials saved to file: [C:\Users\benja\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials w

In [4]:
# Configura la conexión a BigQuery
client = bigquery.Client()

c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
from google.cloud import bigquery

# 1. Inicializa el cliente de BigQuery
client = bigquery.Client()

# 2. Define la consulta SQL completa y correcta
query = """
SELECT 
    id, 
    title, 
    body, 
    source, 
    country, 
    publish_date, 
    platform
FROM 
    `noticias-450720.news_articles.articles` 
WHERE 
    country = 'Chile'
    AND platform = 'Event Registry'
    AND publish_date BETWEEN DATE('2025-04-02') AND DATE('2025-06-29')
"""

# 3. Ejecuta la consulta
print("🚀 Ejecutando la consulta en BigQuery...")
query_job = client.query(query)

# 4. Convierte los resultados a un DataFrame de pandas
print("⏳ Descargando los resultados...")
df_filtrado = query_job.to_dataframe()

# 5. Guarda el DataFrame en un archivo CSV
output_filename = 'datos/noticias_chile_primarias_2025.csv'
df_filtrado.to_csv(output_filename, index=False)

print(f"\n✅ ¡Consulta finalizada con éxito!")
print(f"Se encontraron y guardaron {len(df_filtrado)} noticias.")
print(f"Revisa el archivo: '{output_filename}'")

🚀 Ejecutando la consulta en BigQuery...
⏳ Descargando los resultados...

✅ ¡Consulta finalizada con éxito!
Se encontraron y guardaron 118000 noticias.
Revisa el archivo: 'noticias_chile_primarias_2025.csv'


In [7]:
df.to_csv('data/noticias_combinadas.csv', index=False)

In [11]:
import pandas as pd
import re

# Carga tus datos (ajusta la ruta si es necesario)
df = pd.read_csv('../data/noticias_chile_primarias_2025.csv')

# --- 1. Define tu nueva lista de palabras clave ---
# Puedes agregar o quitar las palabras que necesites.
keywords = [
    'primarias', 'elecciones', 'presidenciales', 'candidato', 'candidata',
    'candidatura', 'votación', 'voto', 'sufragio', 'encuesta', 'debate',
    'pacto electoral', 'servicio electoral', 'servel', 'campaña electoral'
]

# --- 2. Crea un patrón de búsqueda (expresión regular) ---
# Esto crea un patrón como r'\b(primarias|elecciones|...)\b'
# \b asegura que se busquen palabras completas.
pattern = r'\b(' + '|'.join(keywords) + r')\b'

# --- 3. Aplica el filtro usando .str.contains() ---
# .str.contains() es muy rápido para esta tarea.
# case=False: no distingue entre mayúsculas y minúsculas.
# na=False: trata los valores nulos (NaN) como si no contuvieran las palabras.
filtro = df['body'].str.contains(pattern, case=False, na=False)

# Crea el nuevo DataFrame con solo las noticias que cumplen el filtro
df_filtrado = df[filtro].copy()


# --- 4. Revisa los resultados ---
print(f"Noticias originales: {len(df)}")
print(f"Noticias filtradas por palabras clave: {len(df_filtrado)}")

# Muestra una muestra de los títulos de las noticias encontradas
print("\nEjemplos de títulos encontrados:")
print(df_filtrado['title'].head())

C:\Users\benja\AppData\Local\Temp\ipykernel_25712\3552374046.py:24: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro = df['body'].str.contains(pattern, case=False, na=False)


Noticias originales: 118000
Noticias filtradas por palabras clave: 15081

Ejemplos de títulos encontrados:
1     Así quedaron las duplas definitivas de cara a ...
5     Corte Marcial Extraordinaria pone en riesgo a ...
16    Palabra de Honor: Lealtad o Traición | Resumen...
33    "Tiene toda la razón para reclamar": Manuel Pe...
35    Cuesta más de $500 millones: Así es la nueva c...
Name: title, dtype: object


In [13]:
df_filtrado.to_csv('../data/noticias_procesadas_primarias.csv', index=False)